In [2]:
import nibabel as nib
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
from einops.layers.torch import Rearrange
import torch
import torchvision
import scipy

In [43]:
import copy


def normalize_frame(frame):
    """Normalize the frame data to 0-255."""
    frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    return frame.astype(np.uint8)

# Load the image
path = r"C:\Users\razva\Master1\Thesis\Data\ADNI\002_S_4213\Axial_MB_rsfMRI__Eyes_Open_\2019-10-03_10_19_44.0\I1236684_nii\I1236684_Axial_MB_rsfMRI_(Eyes_Open)_20191003101944_11.nii.gz"
img = nib.load(path)
data = img.get_fdata()[:, :, :, :100]
# data = np.zeros(shape=[64, 64, 48, 105])
print(data.shape)

if len(data.shape) < 4:
    data = np.expand_dims(data, axis=-1)

print(data.shape)

TARGET_SPACE_DIM = [64, 64, 48]
zoom_factor = (TARGET_SPACE_DIM[0] / data.shape[0], TARGET_SPACE_DIM[1] / data.shape[1], TARGET_SPACE_DIM[2] / data.shape[2], 1)

TARGET_TIME_DIM = 140

print(data.shape)

crt_space_dim = list(data.shape[:3])
crt_time_dim = data.shape[-1]

if crt_time_dim >= TARGET_TIME_DIM:
    data = data[:, :, :, :TARGET_TIME_DIM]
else:
    padded_data = np.zeros(shape=(crt_space_dim + [TARGET_TIME_DIM - crt_time_dim]))
    print(padded_data.shape)
    data = np.concatenate([data, padded_data], axis=3)

print(data.shape)
    
data = scipy.ndimage.zoom(data, zoom_factor)

print(data.shape)

# Extract shape dynamically
x_dim, y_dim, z_dim, time_stamps = data.shape

# Ensure frame size is even for OpenCV compatibility
frame_size = 256
frame_size = frame_size if frame_size % 2 == 0 else frame_size - 1  

num_cols = int(np.ceil(np.sqrt(time_stamps)))
num_rows = int(np.ceil(time_stamps / num_cols))

dest_folder = "movies"
os.makedirs(dest_folder, exist_ok=True)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

for slice_index in range(x_dim):
    video_path = os.path.join(dest_folder, f"slice_{slice_index}.mp4")
    out = cv2.VideoWriter(video_path, fourcc, 10, (frame_size, frame_size), isColor=True)

    for temp_index in range(time_stamps):
        frame = data[slice_index, :, :, temp_index]
        frame = normalize_frame(frame)

        rot_frame = np.rot90(frame)
        resized_frame = cv2.resize(rot_frame, (frame_size, frame_size))
        
        # Convert grayscale to BGR explicitly
        color_frame = cv2.cvtColor(resized_frame, cv2.COLOR_GRAY2BGR)
        
        out.write(color_frame)  # Ensure frames are written

    out.release()

print("All videos saved successfully.")

(88, 88, 64, 100)
(88, 88, 64, 100)
(88, 88, 64, 100)
(88, 88, 64, 40)
(88, 88, 64, 140)
(64, 64, 48, 140)
All videos saved successfully.


In [44]:
root = r"C:\Users\razva\Master1\Thesis\Data\ADNI"
str = r"C:\Users\razva\Master1\Thesis\Data\ADNI\002_S_4213\Axial_MB_rsfMRI__Eyes_Open_\2019-10-03_10_19_44.0\I1236684_nii"
str.replace(root, "")

'\\002_S_4213\\Axial_MB_rsfMRI__Eyes_Open_\\2019-10-03_10_19_44.0\\I1236684_nii'